# Análisis Profundo del Proyecto CUVS: Introducción y Uso Básico

Este notebook proporciona una introducción profunda al proyecto CUVS de RAPIDS, una biblioteca para búsqueda de vectores y clustering acelerada por GPU. Cubriremos instalación, uso básico y pruebas de utilidad y rendimiento.

## 1. Instalar RAPIDS y CUVS

CUVS es parte de RAPIDS, un suite de bibliotecas para data science acelerada por GPU. Para instalar en Google Colab o un entorno Jupyter, usa pip con soporte CUDA.

Nota: Asegúrate de tener una GPU NVIDIA compatible. En Colab, selecciona Runtime > Change runtime type > GPU.

In [ ]:
# Instalar RAPIDS y CUVS
# Para CUDA 12 (común en Colab)
!pip install cuvs-cu12 --extra-index-url=https://pypi.nvidia.com

# O para CUDA 11
# !pip install cuvs-cu11 --extra-index-url=https://pypi.nvidia.com

# Verificar instalación
import cuvs
print("CUVS versión:", cuvs.__version__)

## 2. Importar Librerías Requeridas

Importamos las librerías necesarias, incluyendo cuVS para operaciones vectoriales, NumPy para arrays, y otras para manejo de datos.

In [ ]:
import numpy as np
import cupy as cp
from cuvs.common import Resources
from cuvs.neighbors import ivf_flat
import time
import matplotlib.pyplot as plt

## 3. Cargar y Preparar Dataset

Cargamos un dataset de ejemplo, como vectores de embeddings. Usaremos datos sintéticos o de sklearn para simplicidad.

In [ ]:
# Generar datos sintéticos: 10000 vectores de 128 dimensiones
np.random.seed(42)
n_samples = 10000
dim = 128
dataset = np.random.randn(n_samples, dim).astype(np.float32)

# Normalizar para similitud coseno
dataset = dataset / np.linalg.norm(dataset, axis=1, keepdims=True)

# Queries: 100 vectores aleatorios
n_queries = 100
queries = np.random.randn(n_queries, dim).astype(np.float32)
queries = queries / np.linalg.norm(queries, axis=1, keepdims=True)

print(f"Dataset shape: {dataset.shape}")
print(f"Queries shape: {queries.shape}")

## 4. Construir Índice CUVS

Creamos un índice usando IVF-Flat, un algoritmo eficiente para búsqueda aproximada de vecinos más cercanos.

In [ ]:
resources = Resources()

# Parámetros de construcción
build_params = ivf_flat.IndexParams(
    n_lists=1024,  # Número de clusters
    metric="cosine",  # Métrica de distancia
    add_data_on_build=True
)

# Construir índice
start_time = time.time()
index = ivf_flat.build(build_params, cp.asarray(dataset), resources=resources)
resources.sync()
build_time = time.time() - start_time

print(f"Índice construido en {build_time:.2f} segundos")
print(f"Índice: {index}")

## 5. Realizar Búsqueda de Vectores

Ejecutamos búsquedas de similitud en el índice construido.

In [ ]:
# Parámetros de búsqueda
search_params = ivf_flat.SearchParams(n_probes=10)  # Número de clusters a buscar
k = 10  # Top-k vecinos

# Búsqueda
start_time = time.time()
distances, neighbors = ivf_flat.search(
    search_params, 
    index, 
    cp.asarray(queries), 
    k=k, 
    resources=resources
)
resources.sync()
search_time = time.time() - start_time

print(f"Búsqueda completada en {search_time:.4f} segundos")
print(f"Tiempo por query: {search_time / n_queries * 1000:.2f} ms")
print(f"Distances shape: {distances.shape}")
print(f"Neighbors shape: {neighbors.shape}")

## 6. Evaluar Precisión de Búsqueda

Calculamos métricas como recall comparando con resultados exactos (usando brute force).

In [ ]:
# Calcular distancias exactas usando brute force
from sklearn.metrics.pairwise import cosine_distances
exact_distances = cosine_distances(queries, dataset)
exact_neighbors = np.argsort(exact_distances, axis=1)[:, :k]

# Calcular recall
def recall_at_k(pred, true, k):
    recall = 0
    for i in range(len(pred)):
        recall += len(set(pred[i]) & set(true[i])) / k
    return recall / len(pred)

approx_neighbors = cp.asnumpy(neighbors)
recall = recall_at_k(approx_neighbors, exact_neighbors, k)
print(f"Recall@{k}: {recall:.4f}")

## 7. Benchmark de Métricas de Rendimiento

Medimos tiempos de construcción, búsqueda y uso de memoria.

In [ ]:
print(f"Tiempo de construcción del índice: {build_time:.2f} s")
print(f"Tiempo total de búsqueda: {search_time:.4f} s")
print(f"Queries por segundo (QPS): {n_queries / search_time:.2f}")

# Uso de memoria (aproximado)
import psutil
process = psutil.Process()
mem_usage = process.memory_info().rss / (1024 ** 3)  # GB
print(f"Uso de memoria aproximado: {mem_usage:.2f} GB")

## 8. Comparar con Alternativas Basadas en CPU

Comparamos con FAISS en CPU para ver la aceleración GPU.

In [ ]:
# Instalar FAISS si no está
!pip install faiss-cpu

import faiss

# Construir índice FAISS
index_faiss = faiss.IndexFlatIP(dim)  # Producto interno para coseno
index_faiss.add(dataset)

# Búsqueda FAISS
start_time_faiss = time.time()
distances_faiss, neighbors_faiss = index_faiss.search(queries, k)
search_time_faiss = time.time() - start_time_faiss

print(f"FAISS búsqueda tiempo: {search_time_faiss:.4f} s")
print(f"CUVS búsqueda tiempo: {search_time:.4f} s")
print(f"Aceleración: {search_time_faiss / search_time:.2f}x")

# Recall FAISS (exacto)
recall_faiss = recall_at_k(neighbors_faiss, exact_neighbors, k)
print(f"Recall FAISS: {recall_faiss:.4f}")
print(f"Recall CUVS: {recall:.4f}")

## 9. Optimizar Parámetros del Índice

Experimentamos con diferentes parámetros como n_lists y n_probes.

In [ ]:
# Experimentar con n_probes
n_probes_list = [1, 5, 10, 20, 50]
recalls = []
times = []

for n_probes in n_probes_list:
    search_params = ivf_flat.SearchParams(n_probes=n_probes)
    start = time.time()
    dist, neigh = ivf_flat.search(search_params, index, cp.asarray(queries), k=k, resources=resources)
    resources.sync()
    t = time.time() - start
    r = recall_at_k(cp.asnumpy(neigh), exact_neighbors, k)
    recalls.append(r)
    times.append(t)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(n_probes_list, recalls, marker='o')
plt.xlabel('n_probes')
plt.ylabel('Recall@10')
plt.title('Recall vs n_probes')

plt.subplot(1, 2, 2)
plt.plot(n_probes_list, times, marker='o')
plt.xlabel('n_probes')
plt.ylabel('Tiempo de búsqueda (s)')
plt.title('Tiempo vs n_probes')
plt.show()

## 10. Manejar Datasets a Gran Escala

Demostramos escalado a datasets más grandes, manejando procesamiento por lotes y memoria GPU.

In [ ]:
# Para datasets grandes, procesar en lotes
large_dataset = np.random.randn(50000, dim).astype(np.float32)
large_dataset = large_dataset / np.linalg.norm(large_dataset, axis=1, keepdims=True)

batch_size = 10000
for i in range(0, len(large_dataset), batch_size):
    batch = large_dataset[i:i+batch_size]
    # Extender índice con batch
    ivf_flat.extend(index, cp.asarray(batch), cp.arange(i, i+len(batch), dtype=cp.int64))
    resources.sync()

print("Índice extendido con datos adicionales")

# Búsqueda en índice grande
distances_large, neighbors_large = ivf_flat.search(
    search_params, index, cp.asarray(queries), k=k, resources=resources
)
resources.sync()
print(f"Búsqueda en dataset grande completada")